## Map Mutations to Protein-Drug Interactions
Map and visualize mutations at protein-drug binding sites.

In [1]:
import os
# Disable Numba: temporary workaround for https://github.com/sbl-sdsc/mmtf-pyspark/issues/288
#os.environ['NUMBA_DISABLE_JIT'] = "1"

In [2]:
from pyspark.sql import SparkSession
from mmtfPyspark.datasets import drugBankDataset, pdbjMineDataset
from ipywidgets import interact, IntSlider, FloatSlider
import py3Dmol
import pandas as pd

In [3]:
# Initialize Spark
spark = SparkSession.builder.appName("4-MapToDrugInteractions").getOrCreate()

2022-01-27 13:31:02 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Enable Arrow-based columnar data transfers between Spark and Pandas dataframes
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#### Input parameters

In [5]:
distance_cutoff = 8  # distance cutoff for visualizing interactions
mw_min = 250  # minimum molecular weight for drug molecules
input_file_name = 'mutations3d_ligand.csv'  # mutations mapped to protein-ligand binding sites
output_file_name = 'mutations3d_drug.csv'  # mutations mapped to protein-drug binding sites

## Read 'mutation3d_ligand.csv' file created in the previous step

In [6]:
df = pd.read_csv(input_file_name)
df['pdbPosition'] = df['pdbPosition'].astype('str') # PDB residue numbers must be string to handle insertion codes
df['queryGroupNumber'] = df['queryGroupNumber'].astype('str')
df.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition,residue,variationId,...,scale,color,queryGroupId,queryChainId,queryGroupNumber,targetGroupId,targetChainId,targetGroupNumber,sequenceIndex,sequence
0,0.750196,"2-acetamido-2-deoxy-beta-D-glucopyranose, 2-ac...",Spike glycoprotein,3.1,6WPS.A,6WPS,333,333,333,"S:p.333T>I(8), S:p.333T>K(4), S:p.333T>A(2)",...,0.163116,#fdd5c4,NAG,A,1305,THR,A,333,351,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
1,0.750196,"2-acetamido-2-deoxy-beta-D-glucopyranose, 2-ac...",Spike glycoprotein,3.1,6WPS.B,6WPS,333,333,333,"S:p.333T>I(8), S:p.333T>K(4), S:p.333T>A(2)",...,0.163116,#fdd5c4,NAG,B,1305,THR,B,333,351,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
2,0.750196,"2-acetamido-2-deoxy-beta-D-glucopyranose, 2-ac...",Spike glycoprotein,3.1,6WPS.E,6WPS,333,333,333,"S:p.333T>I(8), S:p.333T>K(4), S:p.333T>A(2)",...,0.163116,#fdd5c4,NAG,E,1305,THR,E,333,351,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
3,0.736057,"2-acetamido-2-deoxy-beta-D-glucopyranose, 2-ac...",Spike glycoprotein,3.7,6WPT.B,6WPT,333,333,333,"S:p.333T>I(8), S:p.333T>K(4), S:p.333T>A(2)",...,0.163116,#fdd5c4,NAG,B,1305,THR,B,333,351,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
4,0.747840,"2-acetamido-2-deoxy-beta-D-glucopyranose, 2-ac...",Spike glycoprotein,3.7,6WPT.C,6WPT,333,333,333,"S:p.333T>I(8), S:p.333T>K(4), S:p.333T>A(2)",...,0.163116,#fdd5c4,NAG,C,1305,THR,C,333,351,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...


## Get InChiKey for ligands in PDB with molecular weight >= 250

In [7]:
ccQuery = "SELECT c.id as ligand_id, c.formula_weight, d.descriptor as inchi_key FROM pdbj.chem_comp c \
           JOIN cc.pdbx_chem_comp_descriptor d ON d.comp_id = c.id \
           WHERE d.type = 'InChIKey' AND c.formula_weight >= " + str(mw_min)

ligands = pdbjMineDataset.get_dataset(ccQuery).dropDuplicates().toPandas()
ligands.head()

,ligand_id,formula_weight,inchi_key
0,PQQ,330.206,MMXZSJMASHPLLR-UHFFFAOYSA-N
1,PY4,334.262,VRMPGTOTVVJQMU-SNVBAGLBSA-N
2,CRT,596.925,VAZQBTJCYODOSV-RISZBRKMSA-N
3,VS4,621.790,PPIYQXGSPPWVLJ-CONSDPRKSA-N
4,107,449.506,MBXKBJLIESPLIK-UHFFFAOYSA-N


## Join dataset on ligand id to add InchiKeys

In [8]:
df = df.merge(ligands, left_on=['queryGroupId'], right_on=['ligand_id'], how='inner')
df = df.drop_duplicates()
df.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition,residue,variationId,...,queryChainId,queryGroupNumber,targetGroupId,targetChainId,targetGroupNumber,sequenceIndex,sequence,ligand_id,formula_weight,inchi_key
0,0.853888,"2-acetamido-2-deoxy-beta-D-glucopyranose, 2-ac...",Spike glycoprotein,3.6,7NTC.A,7NTC,92,92,92,"S:p.92F>V(12), S:p.92F>L(10), S:p.92F>I(4), S:...",...,A,1301,PHE,A,92,122,MGILPSPGMPALLSLVSLLSVLLMGCVAETGMFVFLVLLPLVSSQC...,BLA,582.646,GWZYPXHJIZCRAJ-SRVCBVSDSA-N
1,0.853888,"2-acetamido-2-deoxy-beta-D-glucopyranose, 2-ac...",Spike glycoprotein,3.6,7NTC.A,7NTC,94,94,94,"S:p.94S>F(3278), S:p.94S>A(66), S:p.94S>T(16),...",...,A,1301,SER,A,94,124,MGILPSPGMPALLSLVSLLSVLLMGCVAETGMFVFLVLLPLVSSQC...,BLA,582.646,GWZYPXHJIZCRAJ-SRVCBVSDSA-N
2,0.853888,"2-acetamido-2-deoxy-beta-D-glucopyranose, 2-ac...",Spike glycoprotein,3.6,7NTC.B,7NTC,94,94,94,"S:p.94S>F(3278), S:p.94S>A(66), S:p.94S>T(16),...",...,B,1301,SER,B,94,124,MGILPSPGMPALLSLVSLLSVLLMGCVAETGMFVFLVLLPLVSSQC...,BLA,582.646,GWZYPXHJIZCRAJ-SRVCBVSDSA-N
3,0.853888,"2-acetamido-2-deoxy-beta-D-glucopyranose, 2-ac...",Spike glycoprotein,3.6,7NTC.A,7NTC,96,96,96,"S:p.96E>D(1412), S:p.96E>G(378), S:p.96E>Q(358...",...,A,1301,GLU,A,96,126,MGILPSPGMPALLSLVSLLSVLLMGCVAETGMFVFLVLLPLVSSQC...,BLA,582.646,GWZYPXHJIZCRAJ-SRVCBVSDSA-N
4,0.853888,"2-acetamido-2-deoxy-beta-D-glucopyranose, 2-ac...",Spike glycoprotein,3.6,7NTC.B,7NTC,96,96,96,"S:p.96E>D(1412), S:p.96E>G(378), S:p.96E>Q(358...",...,B,1301,GLU,B,96,126,MGILPSPGMPALLSLVSLLSVLLMGCVAETGMFVFLVLLPLVSSQC...,BLA,582.646,GWZYPXHJIZCRAJ-SRVCBVSDSA-N


## Download open DrugBank dataset¶

For this demo we use the open DrugBank dataset. One disadvantage of the open DrugBank dataset is that it not only contains approved drugs, but many other compounds in pharmaceutical use such as ethanol, ATP, etc.

In [9]:
drugs = drugBankDataset.get_open_drug_links() \
                       .filter("StandardInChIKey IS NOT NULL") \
                       .toPandas()

DrugBank offers specific datasets for drug groups (ALL, APPROVED, EXPERIMENTAL, NUTRACEUTICAL, ILLLICT, WITHDRAWN, INVESTIGATIONAL) with detailed information (e.g., approval status). To use these datasets, you need to create a free DrugBank account and supply username/passwork in the cell below. 

[Create DrugBank account](https://www.drugbank.ca/public_users/sign_up)

For this demo, we continue with the open drug bank dataset, which has limited information.

In [10]:
# username = "<your DrugBank account username>"
# password = "<your DrugBank account password>"
# drugs = drugBankDataset.get_drug_links("APPROVED", username,password) \
#                        .filter("InChIKey IS NOT NULL") \
#                        .toPandas()
# adjust column names
# drugs = drugs.rename(index=str, columns={"InChIKey": "StandardInChIKey", "Name": "Commonname"})

## Print some sample DrugBank data

In [11]:
drugs.head()

,DrugBankID,AccessionNumbers,Commonname,CAS,UNII,Synonyms,StandardInChIKey
0,DB00006,BIOD00076 | BTD00076 | DB02351 | EXPT03302,Bivalirudin,128270-60-0,TN9BEX005G,Bivalirudin | Bivalirudina | Bivalirudinum,OIRCOABEOLEUMC-GEJPAHFPSA-N
1,DB00007,BIOD00009 | BTD00009,Leuprolide,53714-56-0,EFY6W0M8TG,Leuprorelin | Leuprorelina | Leuproreline | Le...,GFIJNRVAKGFPGQ-LIJARHBVSA-N
2,DB00014,BIOD00113 | BTD00113,Goserelin,65807-02-5,0F65R8P09N,Goserelin | Goserelina,BLCLNMBMMGCOAS-URPVMXJPSA-N
3,DB00027,BIOD00036 | BTD00036,Gramicidin D,1405-97-6,5IE62321P4,Bacillus brevis gramicidin D | Gramicidin | Gr...,NDAYQJDHGXTBJL-MWWSRJDJSA-N
4,DB00035,BIOD00061 | BIOD00112 | BTD00061 | BTD00112,Desmopressin,16679-58-6,ENR1LLB0FP,1-(3-mercaptopropionic acid)-8-D-arginine-vaso...,NFLWUMRGJYTJIN-PNIOQBSNSA-N


In [12]:
df = df.merge(drugs, left_on=['inchi_key'], right_on=['StandardInChIKey'], how='inner')
df.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition,residue,variationId,...,ligand_id,formula_weight,inchi_key,DrugBankID,AccessionNumbers,Commonname,CAS,UNII,Synonyms,StandardInChIKey


### Keep only unique ligands per structure
Here we drop rows with the same structureId and ligandId.

## Save protein-ligand mapping

In [13]:
df.to_csv(output_file_name, index=False)

## View mutations grouped by protein chain
Use the slider to view each protein chain. Turn labels off for an unobstructed view of the mutations. Interacting ligands are rendered as spheres with green carbon atoms.

In [14]:
chains = df.groupby('structureChainId')
print("Chains:", list(chains.groups.keys()))

Chains: []


## View mutations grouped by protein chain

In [15]:
# Setup viewer
def view_grouped_mutations(grouped_df, *args):
    chainIds = list(grouped_df.groups.keys())

    def view3d(show_bio_assembly, show_surface, show_short_label, show_long_label, size, font, logFreq, i):
        group = grouped_df.get_group(chainIds[i])
        
        pdb_id, chain_id = chainIds[i].split('.')
        global viewer1
        viewer1 = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer1.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})

        # highlight chain of interest in blue
        viewer1.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue'}})
   
        # non-polymer style
        viewer1.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
        
        # don't display water molecules
        viewer1.setStyle({'resn': ['HOH','DOD']}, {})
        
        rows = group.shape[0]
        for j in range(0, rows):
            if group.iloc[j]['scale'] > logFreq:
                # interacting residue info
                res_num = str(group.iloc[j]['pdbPosition'])
                mod_res = {'resi': res_num, 'chain': chain_id}
            
                # interacting ligand info
                lig_id = group.iloc[j]['queryGroupId']
                lig_chain = group.iloc[j]['queryChainId']
                lig_num = group.iloc[j]['queryGroupNumber']
                lig_name = df.iloc[i]['Commonname']
                lig_res = {'resi': lig_num, 'chain': lig_chain}
            
                col = group.iloc[j]['color']
                c_col = col + 'Carbon'
                viewer1.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
                viewer1.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.6}})   
                viewer1.addStyle(lig_res, {'sphere': {'colorscheme': 'greenCarbon'}})
            
                annotation = group.iloc[j]['annotation']
                variationId = group.iloc[j]['variationId']
            
                if show_short_label:
                    label = annotation
                if show_long_label:
                    label = chain_id + "." + str(res_num) + ": " + variationId
                if show_short_label or show_long_label:
                    viewer1.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory'}, mod_res)
                    viewer1.addLabel(lig_id + '.' + lig_chain + '.' + lig_num + ': ' + lig_name, {'fontSize':font}, lig_res)        

        viewer1.zoomTo({'chain': chain_id})
        viewer1.center({'chain': chain_id})
        
        if show_surface:
             viewer1.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'},{'chain': chain_id})
                
        # print header
        name = group.iloc[0]['name']
        resolution = group.iloc[0]['resolution']
        coverage = group.iloc[0]['coverage']
        description = group.iloc[0]['description']
        drugBankId = group.iloc[0]['DrugBankID']
        commonName = group.iloc[0]['Commonname']
        
        print(f'protein: {name}, description: {description}')
        print(f'drug: {commonName}, drugBankId: {drugBankId}')
        print()
        print(f'PDB Id: {pdb_id}, chain Id: {chain_id}, resolution: {resolution}, sequence coverage: {coverage:.2f}')
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + group.iloc[j][a])
                
        return viewer1.show()
    
    f_widget = IntSlider(value=9, min=5, max=20, description='font size', continuous_update=False)
    z_widget = IntSlider(value=750, min=500, max=1200, description='size', continuous_update=False)
    s_widget = IntSlider(min=0, max=len(chainIds)-1, description='structure', continuous_update=False)
    l_widget = FloatSlider(value=0.5, min=0, max=1, step=0.05, description='logFreq:', disabled=False, 
                           continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f')
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_short_label=True, show_long_label=False, 
                    size=z_widget, font=f_widget, logFreq=l_widget,  i=s_widget)

def view_image1():
    return viewer1.png()

In [16]:
view_grouped_mutations(chains);

TraitError: setting max < min

## View one mutation at a time
Use the slider to view each mutation. Surrounding residues within the `distance_cutoff` are rendered as orange sticks. Interacting ligands are rendered as spheres with green carbon atoms.

In [ ]:
# Setup viewer
def view_single_mutation(df, distance_cutoff, *args):

    def view3d(show_bio_assembly, show_surface, show_short_label, show_long_label, size, font, logFreq, i):        
        pdb_id, chain_id = df.iloc[i]['structureChainId'].split('.')
        
        global viewer2
        viewer2 = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer2.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.7}})
       
        # highlight chain of interest in blue
        viewer2.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue', 'width': 0.6, 'opacity':0.5}})
        
        # non-polymer style
        viewer2.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
        
        # don't display water molecules
        viewer2.setStyle({'resn': ['HOH','DOD']}, {})
        
        # interacting residue info
        res_num = str(df.iloc[i]['pdbPosition'])
        mod_res = {'resi': res_num, 'chain': chain_id}
        
        # interacting ligand info
        label = df.iloc[i]['variationId']  
        
        lig_id = df.iloc[i]['queryGroupId']
        lig_chain = df.iloc[i]['queryChainId']
        lig_num = df.iloc[i]['queryGroupNumber']
        lig_name = df.iloc[i]['Commonname']
        lig_res = {'resi': lig_num, 'chain': lig_chain}
        lig_label = lig_chain + "." + lig_num + "." + lig_id + ": " + lig_name
        
        
        col = df.iloc[i]['color']
        c_col = col + 'Carbon'
        viewer2.addStyle(mod_res, {'sphere':{'color': col, 'opacity': 0.8}})  # TODO opacity has no effect

        annotation = df.iloc[i]['annotation']
        variationId = df.iloc[i]['variationId']
          
        if show_short_label:
            label = annotation
        if show_long_label:
            label = chain_id + "." + str(res_num) + ": " + variationId
        if show_short_label or show_long_label:
            viewer2.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory'}, mod_res)
            viewer2.addLabel(lig_label, {'fontSize':font}, lig_res)     
            
        # select neigboring residues by distance
        surroundings = {'chain': chain_id, 'resi': res_num, 'byres': True, 'expand': distance_cutoff}
        
        # residues surrounding mutation site
        viewer2.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})

        # interacting ligand style
        viewer2.addStyle(lig_res, {'sphere': {'colorscheme': 'greenCarbon'}})   
        
        if show_surface:
             viewer2.addSurface(py3Dmol.SES, {'opacity':0.8,'color':'lightblue'}, {'chain': chain_id})
         
        viewer2.zoomTo(surroundings)
        viewer2.center(surroundings)
        
        # print header
        name = df.iloc[i]['name']
        drugBankId = df.iloc[i]['DrugBankID']
        
        name = df.iloc[i]['name']
        resolution = df.iloc[i]['resolution']
        coverage = df.iloc[i]['coverage']
        description = df.iloc[i]['description']
        drugBankId = df.iloc[i]['DrugBankID']
        commonName = df.iloc[i]['Commonname']
        
        print(f'protein: {name}, description: {description}')
        print(f'drug: {lig_name}, drugBankId: {drugBankId}')
        print()
        print(f'PDB Id: {pdb_id}, chain Id: {chain_id}, resolution: {resolution}, sequence coverage: {coverage:.2f}')
        print(f'ligand: {lig_id}.{lig_chain}.{lig_num}, mutation: {variationId}')
        
  
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])
                
        return viewer2.show()
    
    f_widget = IntSlider(value=9, min=5, max=20, description='font size', continuous_update=False)
    z_widget = IntSlider(value=750, min=500, max=1200, description='size', continuous_update=False)
    s_widget = IntSlider(min=0, max=len(df)-1, description='structure', continuous_update=False)
    l_widget = FloatSlider(value=0.5, min=0, max=1, step=0.05, description='logFreq:', disabled=False, 
                           continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f')
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_short_label=True, show_long_label=False, size=z_widget, 
                    font=f_widget, logFreq= l_widget, i=s_widget)

def view_image2():
    return viewer2.png()

In [ ]:
view_single_mutation(df, distance_cutoff);

In [ ]:
# Shutdown Spark
spark.stop()